# **Database Creation (TargetMartII)**

## Schema creation for Fact and Dimension table

In [1]:
USE TargetMartII;
GO

IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'dim' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA dim AUTHORIZATION dbo;'
END
;                                                                
GO

IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'f' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA f AUTHORIZATION dbo;'
END
;                                                              

GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.018

## Successfully created schemas if not exist.

# **Creating dim.Calendar Table**

In [2]:

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Calendar')
BEGIN
CREATE TABLE dim.Calendar   
(
    pkCalendar INT NOT NULL,
    DateValue DATE NOT NULL,
    Year INT NOT NULL,
    Quarter INT NOT NULL,
	Qtr VARCHAR(3) NOT NULL,
    Month INT NOT NULL,
    MonthName VARCHAR(10) NOT NULL,
	MonthShort VARCHAR(3) NOT NULL,
    Week INT NOT NULL,
    Day INT NOT NULL,
	DayName VARCHAR(10) NOT NULL,
	DayShort VARCHAR(3) NOT NULL,
    IsWeekday BIT NOT NULL,
	Weekday VARCHAR(3) NOT NULL
);

	ALTER TABLE dim.Calendar
	ADD CONSTRAINT PK_Calendar_Julian PRIMARY KEY(pkCalendar);  

	ALTER TABLE dim.Calendar
    ADD CONSTRAINT UC_Calendar UNIQUE (DateValue);   
END

GO

Commands completed successfully.

Total execution time: 00:00:00.062

Above script determines whether a table named 'Calendar' exists in the database's '**dim**' schema. If not, it generates the '**Calendar**' table, which has date-related columns like Year, Quarter, Month, Day, and so on. Additionally, it generates a primary key constraint on the '**pkCalendar**' column and a unique constraint on the '**DateValue**' column.

# **Creating dim.Employee Table**

In [3]:
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Employees')
BEGIN
CREATE TABLE dim.Employees(       
	EmployeeID int NOT NULL,
	Employee nvarchar(50) NOT NULL,
	Title nvarchar(30) NULL,
	BirthDate datetime NULL,
	HireDate datetime NULL,
	City nvarchar(15) NULL,
	Country nvarchar(15) NULL,
	ReportsTo int NULL
);
	ALTER TABLE dim.Employees
	ADD CONSTRAINT PK_Emp PRIMARY KEY(EmployeeID);  

END

GO

Commands completed successfully.

Total execution time: 00:00:00.069

The above script checks whether a table named '**Employees**' exists in the 'dim' schema. If not, it creates a table in the database called '**Employees**' with columns  'EmployeeID,' 'Employee,' 'Title,' 'BirthDate,' 'HireDate,' 'City,' 'Country,' and 'ReportsTo.' The primary key in this table is the '**EmployeeID**'.

# **Creating dim.Shipper Table**

In [4]:
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Shipper')
BEGIN
CREATE TABLE dim.Shipper       
(
    pkShipId int not null,
	Shipper nvarchar(40) not null,
	Phone nvarchar(24) null
);

	ALTER TABLE dim.Shipper
	ADD CONSTRAINT PK_Ship PRIMARY KEY(pkShipId);  
END

GO

Commands completed successfully.

Total execution time: 00:00:00.060

# **Creating dim.Customer Table**

In [5]:
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Customers')
BEGIN
	CREATE TABLE dim.Customers(       
	pkCustomer int IDENTITY(1000,1) NOT NULL,
	CustomerID nvarchar(5) NOT NULL,
	Customer nvarchar(40) NOT NULL,
	City nvarchar(15) NULL,
	Country nvarchar(15) NULL,
	LoadDate DATE NOT NULL,
	SourceCountry nvarchar(25) NULL
	)
	;

	ALTER TABLE dim.Customers
	ADD CONSTRAINT PK_Customers_LUP PRIMARY KEY(pkCustomer);  

	ALTER TABLE dim.Customers
    ADD CONSTRAINT UC_Customers_ID UNIQUE (CustomerID);  

END

GO

Commands completed successfully.

Total execution time: 00:00:00.017

# **Creating fact.Orderperf Table**

In [6]:

IF EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'f' AND TABLE_NAME = 'OrderPerf')
BEGIN 
	DROP TABLE f.OrderPerf;
END

GO

CREATE TABLE f.OrderPerf(        
	OrderID int NOT NULL,
	fkCalendar int NOT NULL,
	fkCustomer int NOT NULL,
	fkShipper int NULL,
	fkEmployee int NULL,
	Freight money NOT NULL,
	DaysToShip int NULL,
	DaysTilRequired int NULL,
	OrderCount int NOT NULL
);


ALTER TABLE f.OrderPerf
ADD CONSTRAINT PK_ORD PRIMARY KEY(OrderID)  
;

ALTER TABLE f.OrderPerf
ADD CONSTRAINT FK_ORDtoCAL
	FOREIGN KEY (fkCalendar)              
	 REFERENCES  dim.Calendar(pkCalendar) 
;

ALTER TABLE f.OrderPerf
ADD CONSTRAINT FK_ORDtoCUST
	FOREIGN KEY (fkCustomer)              
	 REFERENCES  dim.Customers(pkCustomer) 
;

ALTER TABLE f.OrderPerf
ADD CONSTRAINT FK_ORDtoSHIP
	FOREIGN KEY (fkShipper)  
	 REFERENCES dim.Shipper(pkShipId)
;

ALTER TABLE f.OrderPerf
ADD CONSTRAINT FK_ORDtoEMP
	FOREIGN KEY (fkEmployee)  
	 REFERENCES dim.Employees(EmployeeID)
;

GO
     

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.105

# **Creating fact.ProductPerf Table**

In [7]:
IF EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'f' AND TABLE_NAME = 'ProductPerf')
BEGIN 
	DROP TABLE f.ProductPerf;
END

CREATE TABLE f.ProductPerf(      --Table name
	OrderID int NOT NULL,
	fkOrderDate int NOT NULL,
	fkProductID int NOT NULL,
	fkCustomerID int NOT NULL,
	UnitPrice money NOT NULL,
	Quantity smallint NOT NULL,
	Discount float NOT NULL,
	DiscFlag int NOT NULL,
	LineTotal money NULL
);

ALTER TABLE f.ProductPerf
ADD CONSTRAINT PK_ProdPerf PRIMARY KEY(OrderID, fkProductID)   --Primary keys

ALTER TABLE f.ProductPerf
ADD CONSTRAINT FK_PRODtoCAL
	FOREIGN KEY (fkOrderDate)  --Foreign key
	 REFERENCES dim.Calendar(pkCalendar)
;

ALTER TABLE f.ProductPerf
ADD CONSTRAINT FK_PRODtoPROD
	FOREIGN KEY (fkProductID)   --Foreign key
	 REFERENCES dim.Products(pkProdId)
;

ALTER TABLE f.ProductPerf
ADD CONSTRAINT FK_PRODtoCUST
	FOREIGN KEY (fkCustomerID)   --Foreign key
	 REFERENCES dim.Customers(pkCustomer)
;

Commands completed successfully.

Total execution time: 00:00:00.096